In [1]:
import typing

import numpy as np
import pandas as pd

from datasets import load_dataset

import tqdm 
import requests

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

import cltrier_nlp as nlp

In [2]:
SAMPLE_SIZE: int = 1_000

In [3]:
DATASET = load_dataset("stanfordnlp/imdb").shuffle()

In [4]:
def remote_encoding(batch: typing.List[str]) -> typing.List:
    embeds: typing.List[np.ndarray] = []

    for value in tqdm.tqdm(batch):
    
        try: 
            embed = np.array(requests.post(
                'https://inf.cl.uni-trier.de/embed/',
                json={'prompt': value}
            ).json()["response"])
            
        except Exception as _e:
            display(_e)
            embed = None
        
        embeds.append(embed)

    return embeds

In [5]:
ENCODERS: typing.Dict[str, typing.Dict[str, typing.Callable]] = {
    "tfidf": {
        "engine": (tfidf := TfidfVectorizer()),
        "embed_train": lambda x: tfidf.fit_transform(x),
        "embed_test": lambda x: tfidf.transform(x),
    },
    "tiny transformer (local)": {
        "engine": (transformer := nlp.encoder.Encoder()),
        "embed_train": lambda x: np.stack(
            [embed.detach().numpy() for embed in nlp.encoder.EncoderPooler()(transformer(x), form="sent_cls")]),
        "embed_test": lambda x: np.stack(
            [embed.detach().numpy() for embed in nlp.encoder.EncoderPooler()(transformer(x), form="sent_cls")]),
    },
    "sota transformer (remote)": {
        "embed_train": lambda x: remote_encoding(x),
        "embed_test": lambda x: remote_encoding(x),
    }
}

/home/simon/.cache/pypoetry/virtualenvs/cltrier-nlp-9-505dFL-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
CLASSIFIERS: typing.Dict[str, typing.Callable] = {
    "random_forest": RandomForestClassifier,
    "ada_boost": AdaBoostClassifier,
    "decision_tree": DecisionTreeClassifier,
    "k_neighbors": KNeighborsClassifier,
    "mlp": MLPClassifier,
}

In [7]:
results: typing.List[pd.DataFrame] = []

for encoder_label, encoder in ENCODERS.items():

    embed_train = encoder["embed_train"](DATASET["train"][:SAMPLE_SIZE]["text"])
    embed_test = encoder["embed_test"](DATASET["test"][:SAMPLE_SIZE]["text"])

    results.append(
        pd.json_normalize(
            data=[
                classification_report(
                    DATASET["test"]["label"][:SAMPLE_SIZE],
                    (
                        classifier()
                        .fit(
                                embed_train, 
                                DATASET["train"]["label"][:SAMPLE_SIZE]
                            )
                        .predict(embed_test)
                    ),
                    zero_division=1.,
                    output_dict=True
                ) | {"classifier": classifier_label, "encoder": encoder_label}
                for classifier_label, classifier in CLASSIFIERS.items()
            ]
        )
        .set_index(["encoder", "classifier"], drop=True)
        .filter(
            items=[
                "accuracy",
                "macro avg.f1-score",
                "weighted avg.f1-score"
            ]
        )
        .sort_values(by="accuracy", ascending=False)
    )

/home/simon/.cache/pypoetry/virtualenvs/cltrier-nlp-9-505dFL-py3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/simon/.cache/pypoetry/virtualenvs/cltrier-nlp-9-505dFL-py3.12/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/simon/.cache/pypoetry/virtualenvs/cltrier-nlp-9-505dFL-py3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 1000/1000 [03:08<00:00,  5.31it/s]
/home/simon/.cache/pypoetry/virtualenvs/cltrier-nlp-9-505dFL

In [8]:
pd.concat(results).sort_values(by="weighted avg.f1-score", ascending=False)

accuracy  macro avg.f1-score  \
encoder                   classifier                                    
sota transformer (remote) random_forest     0.885            0.884163   
                          mlp               0.878            0.877875   
                          k_neighbors       0.865            0.864277   
                          ada_boost         0.849            0.848575   
tfidf                     mlp               0.805            0.804223   
                          random_forest     0.801            0.800727   
sota transformer (remote) decision_tree     0.765            0.764338   
tfidf                     ada_boost         0.741            0.737769   
tiny transformer (local)  mlp               0.721            0.720575   
                          random_forest     0.693            0.692483   
tfidf                     decision_tree     0.666            0.665410   
tiny transformer (local)  ada_boost         0.665            0.664126   
                          k_neighbors       0.631            0.629336   
                          decision_tree     0.613            0.612674   
tfidf                     k_neighbors       0.623            0.605780   

                                         weighted avg.f1-score  
encoder                   classifier                            
sota transformer (remote) random_forest               0.884557  
                          mlp                         0.878031  
                          k_neighbors                 0.864673  
                          ada_boost                   0.848896  
tfidf                     mlp                         0.804716  
                          random_forest               0.801022  
sota transformer (remote) decision_tree               0.764838  
tfidf                     ada_boost                   0.738933  
tiny transformer (local)  mlp                         0.721011  
                          random_forest               0.692987  
tfidf                     decision_tree               0.665972  
tiny transformer (local)  ada_boost                   0.664812  
                          k_neighbors                 0.628343  
                          decision_tree               0.613124  
tfidf                     k_neighbors                 0.609076